#### Import Libraries

In [ ]:
import cv2                 
import numpy as np         
import os                  
from random import shuffle
import matplotlib.pyplot as plt
import glob as gb


In [ ]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold


In [ ]:
from keras import backend as K

In [ ]:
from keras_unet.models import custom_unet

#### Setting the dataset path


In [ ]:
#read DataSet
Image_Directory ="./Dataset/"


In [ ]:
#to get all image names in train file
Healthy_images = os.listdir(Image_Directory + "/Healthy_images")
Covid_images = os.listdir(Image_Directory + "/Covid_images")

In [ ]:
Labels={'Covid_images':0,'Healthy_images':1 }

#convert label to code
def getCode(label):
    return Labels[label]

#convert code to label 
def getLabel(n):
    for x,c in Labels.items():
        if n==c:
            return x
               
#Test        
print(getCode('Covid_images'))
print(getLabel(1))

#### Load Data

In [ ]:
#Reading image data
sizeImage=224 # to resize the all image as same size

#to read all images from directory
def getData(Dir,sizeImage):
    X=[]
    y=[]
    for folder in  os.listdir(Dir) : #to get the file name 
        files = gb.glob(pathname= str( Dir  +"/" +folder+ '//*.png' )) #to get the images
        for file in files:
                picture=cv2.imread(file)
                imageArray=cv2.resize(picture,(sizeImage,sizeImage))
                X.append(list(imageArray))
                y.append(getCode(folder))
    X=np.array(X)
    y=np.array(y)
    return X,y

In [ ]:
#get train data
X, y = getData(TrainImage,sizeImage)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print("X_train Shape        ",X_train.shape)

In [ ]:
#Convert y_train to categorical
y_train=to_categorical(y_train,2)
print("y_train ",y_train.shape)

#Convert y_train to categorical
y_test=to_categorical(y_test,2)
print("y_test ",y_test.shape)

In [ ]:
train_datagen = ImageDataGenerator(
      samplewise_center=True,
      samplewise_std_normalization= True,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      fill_mode='nearest')

#### Build Model

In [ ]:
#load weight
Network_Weight="./unet.h5"
print(Network_Weight)

pre_trained_model = custom_unet(input_shape=(224, 224, 3))

for layer in pre_trained_model.layers:
     layer.trainable = False

In [ ]:
x = tf.keras.layers.Flatten()(pre_trained_model.output)

#Full Connected Layers
x = tf.keras.layers.Dense(512, activation='relu')(x)
#Add dropout to avoid Overfit
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
#Add dropout to avoid Overfit
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

x=tf.keras.layers.Dense(2 , activation='sigmoid')(x)   

model = Model(pre_trained_model.input, x) 

model.compile(optimizer='adam', loss="binary_crossentropy",
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

#### K-fold Cross Validation

In [ ]:
#K-fold Cross Validation model evaluation
num_folds = 5
fold_no = 1
epochs = 10
acc_per_fold = []
loss_per_fold = []

kfold = KFold(n_splits=num_folds, shuffle=True, random_state=None)

In [ ]:
lr_reduce = ReduceLROnPlateau(monitor='accuracy', factor=0.1, min_delta=0.0001, patience=1, verbose=1)

filepath="uuuche.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
score_list = []

for train, test in kfold.split(X_train, y_train):
    
    train_generator =train_datagen.flow(
         X_train[train], y_train[train],
         batch_size= 256
    )

    test_generator =train_datagen.flow(
         X_train[test], y_train[test],
         batch_size= 50
    )  

    history = model.fit(train_generator,steps_per_epoch=20,callbacks=[lr_reduce,checkpoint],
             epochs=epochs)

    scores = model.evaluate(test_generator)
    score_list.append(scores)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%; {model.metrics_names[2]} of {scores[2]}')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    fold_no = fold_no + 1

In [ ]:
for s in score_list:
    print(s[1]*100)

#### Model Evaluation

In [ ]:
test_generator =train_datagen.flow(
     X_test, y_test,
     batch_size= 50,
)

In [ ]:
#Evaluate Model
model.evaluate(test_generator)

In [ ]:
#prediction
pred = model.predict(test_generator)

In [ ]:
print("pred \n",len(pred))
print("y_test \n",len(y_test))

print("y_test \n",y_test)
print("pred \n",pred)